# Use this notebook for developing mlframe prototype 2

## Object-oriented approach. Testing on DL with benchmarks.

In [1]:
import multiprocessing
from Queue import Queue, Empty
import subprocess
import os
import time
import re
import sys

In [2]:
def printObj(obj):
    for key,val in obj.__dict__.iteritems():
        print key,"=",val

In [3]:
# Class for executing commands in background processes.
# command_and_args should be an instance of class Command,
# but can also be a list or a string with the command and arguments separated with spaces.
# It can be used for executing remote processes in background. For that purpose exec_remote.sh should be used.
# Usage sample: 
# d = BashExecutor(command,hostname=hostname)
# d.start()
# exec_remote.sh must output subprocess exit code in the form:
# exitcode=N
# , where N is the number.
class BackgroundExecutor(multiprocessing.Process):
    
    def __init__(self, d = {}, debug=False, hostname=""):
        super(BackgroundExecutor,self).__init__()
        self.debug = debug
        self.hostname = hostname
        self.exitcode_pat = re.compile("^exitcode=(\d+)")
        self.d = d
        self.command = d["command"].split(" ")
        if debug:
            print "In ",self.name," command='",self.command,"'"
        
        
    # Poll exit code of self.proc and store it if not None.
    def poll(self):
        exitcode = self.proc.poll()
        if exitcode is not None:
            self.setExitCode(exitcode)
        return exitcode
    
    # Set given exit code to Command class object
    def setExitCode(self, ec):
#         if self.debug:
#             print "Manager.dict object:",repr(self.d)
#             print type(self.d)
#             printObj(self.d)
        if self.d["exitcode"] == "":
            if self.debug: print "setting exit code to",ec,
            self.d["exitcode"] = ec                        
        
            
    def run(self):
        if self.debug: 
            print "In {}. Calling {}".format(self.name,self.command)
            
        command = self.command
        proc = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, bufsize=1, shell=False)
        self.proc = proc
        if self.debug: print self.name, "process started"
        std = ""
        for std in iter(proc.stdout.readline, b''):
            if std is not None and len(std) > 0:
                self.d["stdout"] = self.d["stdout"] + std
                if self.debug:
                    self.d[1]=1                    
                    assert self.d[1] ==1, "Cannot set to Manager dictionary"
                print std,
            
            time.sleep(.5)
        print "Exit code:",self.poll()
    
    def nameYourself(self):
        cp = multiprocessing.current_process()
        print "name",cp._name
        print "parent pid",cp._parent_pid
        print "id",cp._identity
    
    def getExitcode(self):
        return self.d["exitcode"]
            

In [4]:
class BashExecutor:
    def __init__(self, command, debug=False):
        manager = multiprocessing.Manager()
        self.d = manager.dict()
        self.d["command"] = command
        if debug:
            print "Command set to '",self.d["command"],"'"
            print "type:",type(self.d["command"])
        self.d["stdout"] = ""
        self.d["exitcode"] = ""
        self.debug = debug
        self.BE = BackgroundExecutor(self.d, self.debug)
        
    def start(self):
        self.BE.start()
        
    def getExitcode(self):        
        return self.d["exitcode"]
    
    def getStdout(self):
        return self.d["stdout"]
    
    def getCommand(self):
        return self.d["command"]
        
        

In [21]:
# Class for storing host-related data: hostname, access key and username.
# Stores commands (instances of Command class) executed on the host.
# Has methods for connecting to the host with ssh, connection test, executing commands.
class Host(object):
    
    def __init__(self, hostname, address = "localhost", user = "", key = "",debug=False):
        self.hostname = hostname
        self.address = address
        self.user = user
        key = key.replace("~",os.environ['HOME'])
        self.key = key
        self.debug = debug
        self.ssh_command = "ssh"
        if key != "":
            self.ssh_command += "-i "+key
        self.host=""
        if user != "":
            self.host += user+"@"
        self.host += address
        self.commands = []
        if debug:
            print "hostname,address,user,key:",self.hostname,self.address,self.user,self.key
            print "ssh command:",self.ssh_command.replace(' ','.'),"host:",self.host
        
    def ping(self, N=5):        
        comm = BashExecutor("ping -c "+str(N)+" "+self.address, debug=self.debug)
        print "Append:",self.commands.append(comm)
        index = len(self.commands)
        self.commands.append(comm)
        comm.start()        
        return index
        
    
    def execute(self,command,options=""):
        if options != "":
            options = " "+options
        command = self.ssh_command+options+" "+self.host+" "+command
        #command = command.split(" ")
        if self.debug:
            print "Executing:",command.replace(' ','.')
        
        comm = BashExecutor(command, debug=self.debug)
        print "Append:",self.commands.append(comm)
        index = len(self.commands)
        self.commands.append(comm)
        comm.start()        
        return index
            
    
    def connect_test(self):
        options = "-o ConnectTimeout=5"
        return self.execute("hostname",options=options)
        

In [22]:
dl_serv = Host("DL","DL",debug=True)

hostname,address,user,key: DL DL  
ssh command: ssh host: DL


In [24]:
dl_serv.connect_test()

Executing: ssh.-o.ConnectTimeout=5.DL.hostname
Command set to ' ssh -o ConnectTimeout=5 DL hostname '
type: <type 'str'>
In  BackgroundExecutor-14  command=' ['ssh', '-o', 'ConnectTimeout=5', 'DL', 'hostname'] '
Append: None


3

In BackgroundExecutor-14. Calling ['ssh', '-o', 'ConnectTimeout=5', 'DL', 'hostname']
BackgroundExecutor-14 process started
DL-Server
Exit code: setting exit code to 0 0


In [25]:
dl_serv.execute("nvidia-smi dmon -c 2 -s u")

Executing: ssh.DL.nvidia-smi.dmon.-c.2.-s.u
Command set to ' ssh DL nvidia-smi dmon -c 2 -s u '
type: <type 'str'>
In  BackgroundExecutor-16  command=' ['ssh', 'DL', 'nvidia-smi', 'dmon', '-c', '2', '-s', 'u'] '
Append: None


5

In BackgroundExecutor-16. Calling ['ssh', 'DL', 'nvidia-smi', 'dmon', '-c', '2', '-s', 'u']
BackgroundExecutor-16 process started
# gpu    sm   mem   enc   dec
# Idx     %     %     %     %
    0     0     0     0     0
    1     0     0     0     0
    2     0     0     0     0
    3     0     0     0     0
    4     0     0     0     0
    5     0     0     0     0
    6     0     0     0     0
    7     0     0     0     0
    0     0     0     0     0
    1     0     0     0     0
    2     0     0     0     0
    3     0     0     0     0
    4     0     0     0     0
    5     0     0     0     0
    6     0     0     0     0
    7     0     0     0     0
Exit code: setting exit code to 0 0


In [6]:
key = "~/.ssh/id_rsa_com"
mouse = Host("mouse","mouse.local",user="peter",key=key,debug=True)

hostname,address,user,key: mouse mouse.local peter /Users/peterbryzgalov/.ssh/id_rsa_com
ssh command: ssh -i /Users/peterbryzgalov/.ssh/id_rsa_com host: peter@mouse.local


In [7]:
#test = mouse.ping(1)
#test = mouse.connect_test()
test = mouse.execute("hostname && date")

Executing: ssh -i /Users/peterbryzgalov/.ssh/id_rsa_com peter@mouse.local hostname && date
Command set to ' ssh -i /Users/peterbryzgalov/.ssh/id_rsa_com peter@mouse.local hostname && date '
type: <type 'str'>
In  BackgroundExecutor-2  command=' ['ssh', '-i', '/Users/peterbryzgalov/.ssh/id_rsa_com', 'peter@mouse.local', 'hostname', '&&', 'date'] '
Append: None
In BackgroundExecutor-2. Calling ['ssh', '-i', '/Users/peterbryzgalov/.ssh/id_rsa_com', 'peter@mouse.local', 'hostname', '&&', 'date']
BackgroundExecutor-2 process started
mouse
Wed Mar 14 09:50:12 JST 2018
Exit code: setting exit code to 0 0


In [9]:
print test
i=test
print mouse.commands[i].getCommand()
print mouse.commands[i].getStdout()
print mouse.commands[i].getExitcode()

1
ssh -i /Users/peterbryzgalov/.ssh/id_rsa_com peter@mouse.local hostname && date
mouse
Wed Mar 14 09:50:12 JST 2018

0


In [286]:
# Execute command on remote (or local) host in background, using exec_remote.sh and BashExecutor instance.
# Command can be any command or executable file name.
# Arguments must be in a string separated with spaces.
def RemoteExec(host, command_and_args, debug=False):    
    if debug: print "RemoteExec commands and args:",command_and_args
    CA = command_and_args.split(" ")
    if debug: print CA
    command = CA[0]
    args = CA[1:]
    if debug: print "Command:",command
    if debug: print "Args:",args
    package_directory = os.path.dirname(os.getcwd())
    scripts_location=os.path.realpath(os.path.join(package_directory,"mlframe","scripts"))
    exec_remote_script="exec_remote.sh"
    script_path = os.path.join(scripts_location,exec_remote_script)
    
    # Merge args into a string
    command_script_path = os.path.join(scripts_location, command)
    if debug: print "Test if script file exists in RemoteExec",command_script_path
    if os.path.isfile(command_script_path):
        command = command_script_path
    if debug: print "Command:",command
          
    if host == "" or host == "localhost" or host == "127.0.07":
        command_args_list = [command] + args
    else:
        command_args_list = [script_path] + [host] + [command] + args
    if debug: print "Calling BashExecutor with args",command_args_list
    d = BashExecutor(command_args_list, hostname=host, debug=debug)
    d.start()

    #for d in jobs:
    #    d.join()
    
    if debug: print "finished",host,command

In [ ]:
muse = Host("muse","52.158.238.181", user="ubuntu", key=key,debug=False)

In [ ]:
muse.ping()
muse.connect_test()

In [ ]:
reedbush = Host("reedbush","reedbush.cc.u-tokyo.ac.jp",